In [ ]:
!pip install pareto

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import pareto
import math

In [ ]:
dataset = 'anime' # 'ml100k', 'ml1m', 'ft', 'anime'

## Frente de pareto

Load GridSearch results:

In [ ]:
df = {}
for m in ['ResBeMF', 'BeMF', 'DirMF', 'PMF', 'MLP']:
    aux = pd.read_csv('results/gridsearch/' + dataset + '/' + m.lower() +'.csv')
    aux = aux[aux['cummulativemae_avg'] >= 0.05]
    aux = aux[aux['cummulativecoverage_avg'] >= 0.05]
    df[m] = aux

Plot the scatter:

In [ ]:
params = ['numFactors', 'regularization', 'learningRate', 'numIters']
methods = ['ResBeMF', 'BeMF', 'DirMF']
cmaps = ['Blues', 'Oranges', 'Greens']

fig, axs = plt.subplots(len(params), len(methods), figsize=(6*len(methods),3.75*len(params)))
fig.tight_layout(h_pad=4, w_pad=6.5)

for i, param in enumerate(params):
    for j, method in enumerate(methods):
        
        axs[i,j].set_xlabel('coverage', fontsize=17)
        axs[i,j].set_ylabel('1-mae', fontsize=17)
        
        if i == 0:
            axs[i,j].set_title(method, fontsize=20)
            
        axs[i,j].tick_params(axis='both', which='major', labelsize=14)
            
        mae = df[method]['cummulativemae_avg']
        coverage = df[method]['cummulativecoverage_avg']
        color = df[method][param] 
                    
        norm = colors.LogNorm(vmin=color.min(), vmax=color.max()) if param == 'regularization' and (method == 'BeMF' or (method == 'ResBeMF' and dataset == 'anime')) else None
        
        im = axs[i,j].scatter(coverage, mae, c=color, cmap=cmaps[j], norm=norm)
        
        cbar = plt.colorbar(im, ax=axs[i,j])
        cbar.set_label(param, fontsize=15)
        cbar.ax.tick_params(labelsize=14)
        
plt.savefig('./figs/' + dataset + '-hyperparameters.png', dpi=300, bbox_inches='tight')

Plot the pareto fronts:

In [ ]:
def get_pareto_front(df):
    objectives = [df.columns.get_loc(c) for c in ['cummulativemae_avg', 'cummulativecoverage_avg'] if c in df]
    nondominated = pareto.eps_sort([list(df.itertuples(False))], objectives, maximize_all=True)
    pareto_front = pd.DataFrame.from_records(nondominated, columns=list(df.columns.values))
    s = sorted(zip(pareto_front['cummulativecoverage_avg'], pareto_front['cummulativemae_avg']))
    cov, mae = [list(tuple) for tuple in zip(*s)]
    return cov, mae

In [ ]:
resbemf_coverage, resbemf_mae = get_pareto_front(df['ResBeMF'])
plt.plot(resbemf_coverage, resbemf_mae, label='ResBeMF')

bemf_coverage, bemf_mae = get_pareto_front(df['BeMF'])
plt.plot(bemf_coverage, bemf_mae, label='BeMF')

dirmf_coverage, dirmf_mae = get_pareto_front(df['DirMF'])
plt.plot(dirmf_coverage, dirmf_mae, label='DirMF')

pmf_coverage, pmf_mae = get_pareto_front(df['PMF'])
plt.scatter(pmf_coverage, pmf_mae, label='PMF', c='red', marker='+')

mlp_coverage, mlp_mae = get_pareto_front(df['MLP'])
plt.scatter(mlp_coverage, mlp_mae, label='MLP', c='purple', marker='+')

plt.xlabel('coverage', fontsize=17)
plt.xticks(fontsize=14)

plt.ylabel('1-mae', fontsize=17)
plt.yticks(fontsize=14)
    
plt.legend(fontsize=14, loc=(1.04, 0.5))

plt.savefig('./figs/' + dataset + '-pareto-front.png', dpi=300, bbox_inches='tight')

## Test split error

Load test split results:

In [ ]:
mae_df = pd.read_csv('results/test-split/' + dataset + '/mae.csv')
mae_df

In [ ]:
coverage_df = pd.read_csv('results/test-split/' + dataset + '/coverage.csv')
coverage_df

Remove mae values computed with a coverage less than 0.01:

In [ ]:
for c in mae_df.columns.tolist()[1:]:
    for i in range(mae_df['reliability'].size):
        if coverage_df[c][i] <= 0.01:
            mae_df[c][i] = math.nan

Compute aveage error and its std:

In [ ]:
for method in ["ResBeMF", "BeMF", "DirMF"]:
    mae_df[method + "_mean"] = mae_df.loc[:, mae_df.columns.str.startswith(method)].mean(axis=1)
    mae_df[method + "_std"] = mae_df.loc[:, mae_df.columns.str.startswith(method)].std(axis=1)

In [ ]:
for method in ["ResBeMF", "BeMF", "DirMF"]:
    coverage_df[method + "_mean"] = coverage_df.loc[:, coverage_df.columns.str.startswith(method)].mean(axis=1)
    coverage_df[method + "_std"] = coverage_df.loc[:, coverage_df.columns.str.startswith(method)].std(axis=1)

Plot the figure:

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14,7))

def plot_axis(df, name, ax):
    ax.set_xlabel(r'reliability threshold ($\theta$)', fontsize=17)
    ax.set_ylabel(name, fontsize=17)

    ax.tick_params(axis='both', which='major', labelsize=14)

    ax.fill_between(df['reliability'], df['ResBeMF_mean'] + df['ResBeMF_std'], df['ResBeMF_mean'] - df['ResBeMF_std'], color='blue', alpha=0.1)
    ax.plot(df['reliability'], df['ResBeMF_mean'], c='blue', label='ResBeMF')

    ax.fill_between(df['reliability'], df['BeMF_mean'] + df['BeMF_std'], df['BeMF_mean'] - df['BeMF_std'], color='orange', alpha=0.1)
    ax.plot(df['reliability'], df['BeMF_mean'], c='orange', label='BeMF')

    ax.fill_between(df['reliability'], df['DirMF_mean'] + df['DirMF_std'], df['DirMF_mean'] - df['DirMF_std'], color='green', alpha=0.1)
    ax.plot(df['reliability'], df['DirMF_mean'], c='green', label='DirMF')
    
    ax.plot(df['reliability'], df['PMF'], c='red', label='PMF')

    ax.plot(df['reliability'], df['MLP'], c='purple', label='MLP')

    ax.legend(fontsize=14)
    
plot_axis(mae_df, '$1-mae^θ$', axs[0])
plot_axis(coverage_df, '$coverage^θ$', axs[1])

plt.savefig('./figs/' + dataset + '-test-error.png', dpi=300, bbox_inches='tight')